In [1]:
import sys
sys.path.append("../../")
# print(sys.path)

from IJCAI19.test.DefenseTest import *
# InitGlobal()

from attack import *
tf.flags.DEFINE_string(
        'dev_dir', '', 'dev directory with images.')
FLAGS.dev_dir = "../../../official_data/dev_data/"

FLAGS.input_dir = "../../../test_data/NonTargetAttackResult/"
FLAGS.output_file = "../../../test_data/DefenseResult.csv"
FLAGS.weight_path = "../../IJCAI19/weight/"
FLAGS.batch_size = 8

ModelFactory.WEIGHT_DIR = FLAGS.weight_path
TARGET_ATTACK = False

from IJCAI19.model.RandomDefense import RandomDefense
 

Using TensorFlow backend.


In [6]:
#Keras test
batch_shape = [FLAGS.batch_size, FLAGS.image_height, FLAGS.image_width, 3]
name = "keras_xception_19"
D = ModelFactory.create(name=name, nb_classes=FLAGS.num_classes)

img_loader = ImageLoader(FLAGS.input_dir, batch_shape, targetlabel=False, label_size=FLAGS.num_classes, format='png', label_file=None)
D.predict_create_graph(batch_shape, use_prob=False)
with open(FLAGS.output_file, 'w') as out_file:
    for filenames, X, _ in img_loader:
        ypred = D.predict_batch(X, None)
        for filename, label in zip(filenames, ypred):
            out_file.write('{0},{1}\n'.format(filename, label))
            # print(filename, label)

Validate()
D.clear_session()

loaded keras model from  ../../IJCAI19/weight/xception_19/keras_xception_19.h5
correct 86/110,  Score 13.445609953002126


In [2]:
batch_shape = [FLAGS.batch_size, FLAGS.image_height, FLAGS.image_width, 3]
name = "inception_v1"
T1 = RandomDefense(FLAGS.num_classes, name=name)
T1.random(
        msb_max=16, msb_rate=1, 
        rotation_range=20,
        width_shift_range=0.05,
        height_shift_range=0.05,
#         shear_range=0.05
)
name = "resnetv1_50"
T2 = RandomDefense(FLAGS.num_classes, name=name)
T2.random(
        msb_max=16, msb_rate=1, 
        rotation_range=20,
        width_shift_range=0.05,
        height_shift_range=0.05,
#         shear_range=0.05
)
name = "vgg_16"
T3 = RandomDefense(FLAGS.num_classes, name=name)
T3.random(
        msb_max=16, msb_rate=1, 
        rotation_range=20,
        width_shift_range=0.05,
        height_shift_range=0.05,
#         shear_range=0.05
)


D = EmbeddedDefenseModel("")
D.add_model(T1, weight=1)
#     D.add_model(T2, weight=1)
#     D.add_model(T3, weight=1)

defense(D, repeat=2)

Validate()


INFO:tensorflow:Restoring parameters from ../../IJCAI19/weight/inception_v1/inception_v1.ckpt
